In [1]:
# Import dependencies
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py

## Data Processing

In [2]:
# Read in team pitching logs
pitching_df = pd.read_csv('Resources/Data/pitching_logs.csv')
pitching_df.head()

,Rk,Gtm,Date,At,Opp,Rslt,IP,H,R,ER,UER,BB,SO,HR,HBP,ERA
0,1,1,Apr 8,NaN,BOS,"W,6-5",11.0,9,5,4,1,3,11,1,1,3.27
1,2,2,Apr 9,NaN,BOS,"W,4-2",9.0,5,2,2,0,4,12,1,0,2.70
2,3,3,Apr 10,NaN,BOS,"L,3-4",9.0,5,4,4,0,4,8,1,1,3.10
3,4,4,Apr 11,NaN,TOR,"L,0-3",9.0,10,3,3,0,0,10,1,1,3.08
4,5,5,Apr 12,NaN,TOR,"W,4-0",9.0,5,0,0,0,0,10,0,0,2.49


In [3]:
# Convert Date string to datetime
pitching_df['Date'] = pitching_df['Date'] + ' 2022'
pitching_df['Date'] = pd.to_datetime(pitching_df['Date'], format="%b %d %Y")
pitching_df.head()

,Rk,Gtm,Date,At,Opp,Rslt,IP,H,R,ER,UER,BB,SO,HR,HBP,ERA
0,1,1,2022-04-08,NaN,BOS,"W,6-5",11.0,9,5,4,1,3,11,1,1,3.27
1,2,2,2022-04-09,NaN,BOS,"W,4-2",9.0,5,2,2,0,4,12,1,0,2.70
2,3,3,2022-04-10,NaN,BOS,"L,3-4",9.0,5,4,4,0,4,8,1,1,3.10
3,4,4,2022-04-11,NaN,TOR,"L,0-3",9.0,10,3,3,0,0,10,1,1,3.08
4,5,5,2022-04-12,NaN,TOR,"W,4-0",9.0,5,0,0,0,0,10,0,0,2.49


In [4]:
# Filter for necessary columns
pitching_df = pitching_df[['Rk', 'Gtm', 'Date', 'Opp', 'Rslt', 'R', 'ERA']]
pitching_df.head()

,Rk,Gtm,Date,Opp,Rslt,R,ERA
0,1,1,2022-04-08,BOS,"W,6-5",5,3.27
1,2,2,2022-04-09,BOS,"W,4-2",2,2.70
2,3,3,2022-04-10,BOS,"L,3-4",4,3.10
3,4,4,2022-04-11,TOR,"L,0-3",3,3.08
4,5,5,2022-04-12,TOR,"W,4-0",0,2.49


## Viz

In [5]:
# Create variables for use in chart

# data variables
x = pitching_df['Date']
y = pitching_df['ERA']


# hover label variables
runs = pitching_df['Rslt'].apply(lambda x: int(x[2:4]) if (x[4] == '-') else int(x[2]))
runs_allowed = pitching_df['R']
opponent = pitching_df['Opp']
era_text = ["{:.2f}".format(era) for era in pitching_df['ERA']]

hover = [
    f'NYY {r} - {ra} {opp}<br>   <b>ERA: {era}</b>'
    for r, ra, opp, era in
    zip(runs, runs_allowed, opponent, era_text)
]


# color variables
navy = '#0C2340'
gray = '#C4CED3'

In [6]:
# Create traces
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x, y=y, mode='lines', name='ERA', line_color='white',
        customdata=hover, hovertemplate='%{customdata}<extra></extra>'
))


# Add traces for annotations and text for end of lines
for i, d in enumerate(fig.data):
    fig.add_trace(go.Scatter(x=[d.x[-1]],
                             y=[d.y[-1]],
                             mode='markers+text',
                             text=['<b>' + str(y) + '</b>' for y in [d.y[-1]]],
                             textfont = dict(
                                 size=18,
                                 color='white'),
                             textposition='middle right',
                             marker = dict(
                                 size=24,
                                 color = d.line.color,
                                 line=dict(
                                     width=3,
                                     color=navy)),
                             hoverinfo='skip',
                             showlegend=False))


# Format traces
fig.update_traces(
    line=dict(
        shape='spline',
        smoothing=1.3,
        width=14)
)


# Format axes
fig.update_xaxes(
    dtick='M1',
    tickformat='<b>%b</b>',
    ticklabelmode='period',
    tickfont=dict(
        size=24,
        color=gray),
    gridcolor='rgb(20,45,75)',
    gridwidth=2,
    hoverformat='<b>%b %d</b>'
)

fig.update_yaxes(
    title=dict(
        text="<br><b>Team ERA</b>",
        font=dict(
            size=26,
            color='white')),
    tickfont=dict(
        size=20,
        color=gray),
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['2.00','2.25','2.50','2.75','3.00','3.25','3.50']],
    tickvals=[2, 2.25, 2.5, 2.75, 3, 3.25, 3.5],
    zerolinecolor='rgb(20,45,75)',
    zerolinewidth=2,
    gridcolor='rgb(20,45,75)',
    gridwidth=2
)


# Format layout
fig.update_layout(
    title=dict(
        text="<b>Yankees Team ERA</b><br>Time to Blame Blake?",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(
            size=30,
            color='white')),
    xaxis_range=['2022-04-01','2022-08-18'],
    yaxis_range=[1.9,3.6],
    showlegend=False, 
    hovermode='x',
    hoverlabel=dict(
        bgcolor=navy,
        font_size=16),
    paper_bgcolor=navy,
    plot_bgcolor=navy,
    margin=dict(t=110, b=70, pad=3)
)

py.iplot(fig, filename='Yankees-Team-ERA')